Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from time import time

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.635022
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.259852
Minibatch accuracy: 62.5%
Validation accuracy: 56.2%
Minibatch loss at step 100: 1.971450
Minibatch accuracy: 50.0%
Validation accuracy: 70.0%
Minibatch loss at step 150: 0.852386
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 200: 0.873928
Minibatch accuracy: 68.8%
Validation accuracy: 76.1%
Minibatch loss at step 250: 1.074879
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.595977
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 350: 0.247401
Minibatch accuracy: 93.8%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.744546
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 450: 0.211857
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.554991
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

keep_prob = .5

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.569634
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.532726
Minibatch accuracy: 37.5%
Validation accuracy: 51.2%
Minibatch loss at step 100: 1.731390
Minibatch accuracy: 50.0%
Validation accuracy: 64.7%
Minibatch loss at step 150: 0.781198
Minibatch accuracy: 75.0%
Validation accuracy: 75.5%
Minibatch loss at step 200: 0.740890
Minibatch accuracy: 75.0%
Validation accuracy: 72.5%
Minibatch loss at step 250: 1.112218
Minibatch accuracy: 75.0%
Validation accuracy: 78.5%
Minibatch loss at step 300: 0.547654
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 350: 0.189125
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 400: 0.799832
Minibatch accuracy: 68.8%
Validation accuracy: 81.0%
Minibatch loss at step 450: 0.254156
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.437303
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [32]:
from math import sqrt
batch_size = 128
patch_size = 5
patch_size_2 = 4
depth = 16

num_hidden_1 = 512
num_hidden_2 = 256
num_hidden_3 = 64

decay_rate=.95
learnRate=.05
keepProb=.5

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    tf_learn_rate = tf.placeholder(tf.float32)
    tf_keep_prob  = tf.placeholder(tf.float32)

    # Variables.
    layerconv1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layerconv1_biases = tf.Variable(tf.zeros([depth]))
    print("layer1convshape", layerconv1_weights.get_shape())
    layerconv2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth*2], stddev=0.1))
    layerconv2_biases = tf.Variable(tf.constant(1.0, shape=[depth*2]))
    print("layer2convshape", layerconv2_weights.get_shape())
    #layerconv3_weights = tf.Variable(tf.truncated_normal(
    #  [patch_size_2, patch_size_2, depth, depth], stddev=0.1))
    #layerconv3_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    #print("layer3convshape", layerconv3_weights.get_shape())
   

    layerhidden1_weights = tf.Variable(tf.truncated_normal(
      [1568, num_hidden_1], stddev=.05))
      #[image_size //patch_size_2  * image_size // patch_size_2 * depth, num_hidden_1], stddev=.05))
    layerhidden1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_1]))
    layerhidden2_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_1, num_hidden_2], stddev=.05))
    layerhidden2_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
    layerhidden3_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_2, num_hidden_3], stddev=.05))
    layerhidden3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_3]))
    layerhidden4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_3, num_labels], stddev=.05))
    layerhidden4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def convNet(data,weights, biases,stride=[1, 1, 1, 1]):
        conv = tf.nn.conv2d(data,  weights, stride, padding='SAME')
        hidden = tf.nn.relu(conv +  biases)
        pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        return pool
    
    def model(data, dropout=False):
        LeNet1=convNet(data,layerconv1_weights, layerconv1_biases)
        print("LeNet1shape:", LeNet1.get_shape().as_list())
        LeNet2=convNet(LeNet1,layerconv2_weights, layerconv2_biases)
        print("LeNet2shape:", LeNet2.get_shape().as_list())
        #LeNet3=convNet(LeNet2,layerconv3_weights, layerconv3_biases)
        #print("LeNet3shape:", LeNet3.get_shape().as_list())
        conv=LeNet2
            
        shape = conv.get_shape().as_list()
        print("hidden1shape",layerhidden1_weights.get_shape().as_list())
        reshape = tf.reshape(conv, [shape[0], shape[1] * shape[2] * shape[3]])
        print("after reshape",reshape.get_shape().as_list())
        
        #fullyconnected 1
        hidden = tf.nn.relu(tf.matmul(reshape, layerhidden1_weights) + layerhidden1_biases)
        if dropout==True:
            hidden=tf.nn.dropout(hidden,tf_keep_prob )
        #fullyconnected 2
        hidden = tf.nn.relu(tf.matmul(hidden, layerhidden2_weights) + layerhidden2_biases)
        if dropout==True:
            hidden=tf.nn.dropout(hidden,tf_keep_prob+(1-tf_keep_prob)/2 )
        #fullyconnected 3
        hidden = tf.nn.elu(tf.matmul(hidden, layerhidden3_weights) + layerhidden3_biases)
        # if dropout==True:
        #    hidden=tf.nn.dropout(hidden,tf_keep_prob )
            
        #fullyconnected 4
        return tf.matmul(hidden, layerhidden4_weights) + layerhidden4_biases
  
    # Training computation.
    logits = model(tf_train_dataset, dropout=True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))#+(
#                                                        layerconv1_biases*tf.nn.l2_loss(layerconv1_weights))
#                                                        layerconv2_biases*tf.nn.l2_loss(layerconv2_weights))
#                                                        layerhidden1_biases*tf.nn.l2_loss(layerhidden1_weights)+
#                                                        layerhidden2_biases*tf.nn.l2_loss(layerhidden2_weights)+
#                                                        layerhidden3_biases*tf.nn.l2_loss(layerhidden3_weights)+
#                                                        layerhidden4_biases*tf.nn.l2_loss(layerhidden4_weights))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(tf_learn_rate, global_step,batch_size, decay_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
#    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)


    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

layer1convshape (5, 5, 1, 16)
layer2convshape (5, 5, 16, 32)
LeNet1shape: [128, 14, 14, 16]
LeNet2shape: [128, 7, 7, 32]
hidden1shape [1568, 512]
after reshape [128, 1568]
LeNet1shape: [10000, 14, 14, 16]
LeNet2shape: [10000, 7, 7, 32]
hidden1shape [1568, 512]
after reshape [10000, 1568]
LeNet1shape: [10000, 14, 14, 16]
LeNet2shape: [10000, 7, 7, 32]
hidden1shape [1568, 512]
after reshape [10000, 1568]


In [ ]:
num_steps = 150001
testAccuracy=[]

startTime0 = time()
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_learn_rate:learnRate, tf_keep_prob:keepProb}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
        if (step % 500 == 0): 
            timeDelta=round((time()-startTime0)/60, 3)
            testAccuracy.append( accuracy(test_prediction.eval(), test_labels))
            print('--Test accuracy: %.1f%%, Runtime:' % testAccuracy[-1],timeDelta)
    print('\nFinal Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

timeDelta=round((time()-startTime0)/60, 3)
print("Total runtime:{} min".format(timeDelta))

Initialized
Minibatch loss at step 0: 2.399637
Minibatch accuracy: 17.2%
Validation accuracy: 7.9%
--Test accuracy: 7.9%, Runtime: 0.148


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

plt.figure(figsize=(14,6))
plt.plot(testAccuracy[0:],label="Test Accuracy")
plt.xlabel('Run')
plt.ylabel('Test Accuracy')
plt.legend()
plt.title("Test Accuracy per step(*500)")
plt.show()
plt.close()

print("test accuracy:")
for t in testAccuracy:
    print(t)


## Tracking
### last run : 
 broken.
### best run: 
 96.8 (dropout, weight intialization, batch size, num_hidden, 100001 run, extra hidden layer)

#EOF